In [356]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/avikhalder2005/day1-db/youtube.db


In [357]:
pip install google-api-python-client pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [359]:
from googleapiclient.discovery import build
import pandas as pd
import sqlite3
from datetime import datetime, timezone

API_KEY = os.getenv("YOUTUBE_API_KEY")

youtube = build("youtube", "v3", developerKey=API_KEY)

request = youtube.videos().list(
    part="snippet,statistics",
    chart="mostPopular",
    regionCode="IN",
    maxResults=100
)

response = request.execute()

data = []
now = datetime.now(timezone.utc)

for item in response["items"]:
    stats = item["statistics"]
    snippet = item["snippet"]

    data.append({
        "video_id": item["id"],
        "title": snippet["title"],
        "channel": snippet["channelTitle"],
        "category": snippet["categoryId"],
        "published_at": snippet["publishedAt"],
        "views": int(stats.get("viewCount", 0)),
        "likes": int(stats.get("likeCount", 0)),
        "comments": int(stats.get("commentCount", 0)),
        "collected_at": now
    })

df = pd.DataFrame(data)

print("Rows fetched:", len(df))
df.head()


Rows fetched: 50


,video_id,title,channel,category,published_at,views,likes,comments,collected_at
0,TXwiQZI_1bE,Mahadev Ki Shadi Hai | Hansraj Raghuwanshi | M...,Hansraj Raghuwanshi,10,2026-02-14T00:30:07Z,6795661,100718,4071,2026-02-16 17:51:49.789799+00:00
1,4Rs9I7h53bw,Ramayana Big Update and Jana Nayagan TOTAL LOS...,YOGI BOLTA HAI,24,2026-02-15T04:57:48Z,392287,17589,567,2026-02-16 17:51:49.789799+00:00
2,8lO_GYI8K4o,LIVING IN A FOREST AT NIGHT GONE WRONG!,Techno Gamerz,20,2026-02-15T03:30:11Z,3216313,162992,18743,2026-02-16 17:51:49.789799+00:00
3,3DYZdSzb_RQ,Har Har Mahadev 🔱 Nonstop Mahashivratri Songs ...,Zee Music Company,10,2026-02-14T12:30:04Z,1117822,8694,541,2026-02-16 17:51:49.789799+00:00
4,6X_g9Vo97CU,Band Melam Movie Valentine’s Day Special Tease...,Hash Cinemas,24,2026-02-14T08:35:45Z,102917,2046,24,2026-02-16 17:51:49.789799+00:00


In [360]:
# import os
# import shutil

# INPUT_DB = "/kaggle/input/datasets/avikhalder2005/day1-db/youtube.db"
# WORK_DB = "/kaggle/working/youtube.db"

# if os.path.exists(INPUT_DB):
#     shutil.copy(INPUT_DB, WORK_DB)
#     print("Loaded previous DB")
# else:
#     print("No previous DB found. Creating new.")



import os
import shutil

INPUT_DB = "/kaggle/input/datasets/avikhalder2005/day1-db/youtube.db"
WORK_DB = "/kaggle/working/youtube.db"

# Only copy INPUT_DB if WORK_DB doesn't exist YET
if not os.path.exists(WORK_DB) and os.path.exists(INPUT_DB):
    shutil.copy(INPUT_DB, WORK_DB)
    print("✓ Loaded previous DB from input")
elif os.path.exists(WORK_DB):
    print("✓ Using existing WORK_DB (not overwriting)")
else:
    print("⚠ No database found. Creating new.")


✓ Using existing WORK_DB (not overwriting)


In [361]:
# import sqlite3
# import pandas as pd

# db_path = "/kaggle/input/datasets/avikhalder2005/day1-db/youtube.db"   # adjust name if needed
conn = sqlite3.connect(WORK_DB)

pd.read_sql("SELECT COUNT(*) FROM video_stats;", conn)


,COUNT(*)
0,1851


In [362]:
conn = sqlite3.connect(WORK_DB)


In [363]:
# df.to_sql("video_stats", conn, if_exists="append", index=False)
# Check existing video IDs
existing_ids = set(pd.read_sql("SELECT DISTINCT video_id FROM video_stats", conn)['video_id'].tolist())

# Filter out duplicates
df_new = df[~df['video_id'].isin(existing_ids)]

# Only append new unique videos
if len(df_new) > 0:
    df_new.to_sql("video_stats", conn, if_exists="append", index=False)


In [364]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(WORK_DB)

df_check = pd.read_sql("SELECT * FROM video_stats", conn)

conn.close()

print("Rows in DB:", len(df_check))
df_check.head()


Rows in DB: 1851


,video_id,title,channel,category,published_at,views,likes,comments,collected_at,source
0,rK0S9sEN9-c,Enakena Yaarum Illaye Promo | LIK | Pradeep Ra...,Sony Music South,10,2026-02-13T11:55:13Z,424938,42930,1073,2026-02-13 18:43:03.556957,mostPopular
1,1J1FAqavUwA,Swayambhu - Epic Teaser (Telugu) | Nikhil Sidd...,Junglee Music Telugu,10,2026-02-11T06:04:31Z,7945859,81072,1049,2026-02-13 18:43:03.556957,mostPopular
2,TsgfhAj2YJQ,[Hindi] BGIS 2026 | ROUND 3 | Day 2,KRAFTON INDIA ESPORTS,20,2026-02-13T14:43:05Z,939955,23123,52,2026-02-13 18:43:03.556957,mostPopular
3,QWAohJdKVLA,SR 1010 - Aslam Singer Mewati || मर्द को दर्जा...,Aslam Singer Zamidar,10,2026-02-13T03:30:11Z,305642,33971,2447,2026-02-13 18:43:03.556957,mostPopular
4,HlZ27-MvSaE,CID Bahu | सी.आई.डी बहू | Official Trailer | A...,Bhojpuri Cinema TV Channel,24,2026-02-12T01:30:17Z,425664,3063,71,2026-02-13 18:43:03.556957,mostPopular


In [365]:
import sqlite3

conn = sqlite3.connect(WORK_DB)

query = """
SELECT * FROM video_stats
"""

df = pd.read_sql(query, conn)
conn.close()

print(df.shape)
df.head()


(1851, 10)


,video_id,title,channel,category,published_at,views,likes,comments,collected_at,source
0,rK0S9sEN9-c,Enakena Yaarum Illaye Promo | LIK | Pradeep Ra...,Sony Music South,10,2026-02-13T11:55:13Z,424938,42930,1073,2026-02-13 18:43:03.556957,mostPopular
1,1J1FAqavUwA,Swayambhu - Epic Teaser (Telugu) | Nikhil Sidd...,Junglee Music Telugu,10,2026-02-11T06:04:31Z,7945859,81072,1049,2026-02-13 18:43:03.556957,mostPopular
2,TsgfhAj2YJQ,[Hindi] BGIS 2026 | ROUND 3 | Day 2,KRAFTON INDIA ESPORTS,20,2026-02-13T14:43:05Z,939955,23123,52,2026-02-13 18:43:03.556957,mostPopular
3,QWAohJdKVLA,SR 1010 - Aslam Singer Mewati || मर्द को दर्जा...,Aslam Singer Zamidar,10,2026-02-13T03:30:11Z,305642,33971,2447,2026-02-13 18:43:03.556957,mostPopular
4,HlZ27-MvSaE,CID Bahu | सी.आई.डी बहू | Official Trailer | A...,Bhojpuri Cinema TV Channel,24,2026-02-12T01:30:17Z,425664,3063,71,2026-02-13 18:43:03.556957,mostPopular


In [366]:
df["collected_at"] = pd.to_datetime(
    df["collected_at"],
    format="ISO8601",
    utc=True
)

df["published_at"] = pd.to_datetime(
    df["published_at"],
    format="ISO8601",
    utc=True
)



df = df.sort_values(["video_id", "collected_at"])
df.reset_index(drop=True, inplace=True)



In [367]:
df["views_prev"] = df.groupby("video_id")["views"].shift(1)
df["likes_prev"] = df.groupby("video_id")["likes"].shift(1)
df["comments_prev"] = df.groupby("video_id")["comments"].shift(1)

df["time_prev"] = df.groupby("video_id")["collected_at"].shift(1)


In [368]:
df["delta_hours"] = (
    (df["collected_at"] - df["time_prev"]).dt.total_seconds() / 3600
)


In [369]:
df = df[df["delta_hours"] > 0]


In [370]:
df["delta_views"] = df["views"] - df["views_prev"]
df["delta_likes"] = df["likes"] - df["likes_prev"]
df["delta_comments"] = df["comments"] - df["comments_prev"]


In [371]:
df["view_velocity"] = df["delta_views"] / df["delta_hours"]
df["like_velocity"] = df["delta_likes"] / df["delta_hours"]
df["comment_velocity"] = df["delta_comments"] / df["delta_hours"]


In [372]:
df["engagement_rate"] = (df["likes"] + df["comments"]) / df["views"]
df["engagement_rate"] = df["engagement_rate"].fillna(0)


In [373]:
df["age_hours"] = (
    (df["collected_at"] - df["published_at"]).dt.total_seconds() / 3600
)


In [374]:
df["velocity_prev"] = df.groupby("video_id")["view_velocity"].shift(1)

df["acceleration"] = df["view_velocity"] - df["velocity_prev"]


In [375]:
df["title_length"] = df["title"].str.len()
df["title_word_count"] = df["title"].str.split().apply(len)


In [376]:
df = df.dropna(subset=[
    "view_velocity",
    "acceleration",
    "engagement_rate",
    "age_hours"
])


In [377]:
df[[
    "video_id",
    "view_velocity",
    "acceleration",
    "engagement_rate",
    "age_hours"
]].describe()


,view_velocity,acceleration,engagement_rate,age_hours
count,199.000000,199.000000,199.000000,199.000000
mean,13894.301469,-3840.528486,0.041591,35.504137
std,29577.307141,40837.854547,0.032091,34.920297
min,0.000000,-198443.922553,0.001651,1.352158
25%,0.000000,-14144.200342,0.013532,10.486201
50%,1687.239507,0.000000,0.034746,28.743269
75%,13146.281063,7089.212954,0.059643,49.212607
max,189699.943973,183806.892718,0.126656,179.591659


In [378]:
# Remove negative ages
df = df[df["age_hours"] >= 0]

# Remove negative growth (can happen due to API inconsistencies)
df = df[df["view_velocity"] >= 0]

# Remove extreme unrealistic spikes (outliers)
upper_limit = df["view_velocity"].quantile(0.99)
df = df[df["view_velocity"] <= upper_limit]

df.shape


(197, 27)

In [379]:
df["acceleration"] = df["acceleration"].fillna(0)
df["engagement_rate"] = df["engagement_rate"].fillna(0)


In [380]:
conn = sqlite3.connect(WORK_DB)

df.to_sql("video_features", conn, if_exists="append", index=False)

conn.close()

print("video_features table created")


video_features table created


In [381]:
df_latest = (
    df.sort_values("collected_at")
      .groupby("video_id")
      .tail(1)
)

df_latest.shape


(50, 27)

In [382]:
from sklearn.preprocessing import MinMaxScaler

features = [
    "view_velocity",
    "acceleration",
    "engagement_rate",
    "age_hours"
]

scaler = MinMaxScaler()

df_latest[[f + "_norm" for f in features]] = scaler.fit_transform(df_latest[features])


In [383]:
df_latest["trend_score"] = (
    0.5 * df_latest["view_velocity_norm"] +
    0.2 * df_latest["acceleration_norm"] +
    0.2 * df_latest["engagement_rate_norm"] +
    0.1 * (1 - df_latest["age_hours_norm"])
)


In [384]:
top_trending = df_latest.sort_values("trend_score", ascending=False)

top_trending[[
    "video_id",
    "title",
    "channel",
    "view_velocity",
    "engagement_rate",
    "trend_score"
]].head(10)


,video_id,title,channel,view_velocity,engagement_rate,trend_score
528,Gcf_q4B3Crc,Asla (Official Video) Mankirt Aulakh | Zehr Vi...,Mankirt Aulakh,148747.356931,0.016861,0.809095
782,P87Exm0VMrI,Bridgerton Season 4 | Part 2 Official Trailer ...,Netflix,89634.167486,0.064347,0.699536
490,EriEensiy0I,Poppy Playtime: Chapter 5 - Official Game Trailer,Mob Entertainment,58942.970327,0.076185,0.576581
747,OhHvHrDBru8,Untitled John Wick Game - Announce Trailer | P...,PlayStation,46760.802056,0.077482,0.541935
1412,kbyUZhheUSQ,Untitled John Wick Game - Official Reveal Trai...,IGN,42440.916858,0.057794,0.491732
835,QWAohJdKVLA,SR 1010 - Aslam Singer Mewati || मर्द को दर्जा...,Aslam Singer Zamidar,11988.946569,0.113234,0.450604
1439,kzpS-A3QJqE,Pallichattambi - Kaattuchembakam Video Song | ...,Sony Music Malayalam,42685.894373,0.032711,0.446798
1662,rK0S9sEN9-c,Enakena Yaarum Illaye Promo | LIK | Pradeep Ra...,Sony Music South,50104.227582,0.089797,0.405649
255,7V-WtHURSJM,How I Found a MYSTERIOUS ISLAND in This Minecr...,SenpaiExtras,20252.624724,0.058589,0.401517
1558,nYntKUcSMh4,Legend New Movie Trailer Reaction🔥🤣,Vinay Vines,487.654772,0.126656,0.399833


In [385]:
df_latest["trend_score"].describe()


count    50.000000
mean      0.326739
std       0.125792
min       0.140401
25%       0.244066
50%       0.293746
75%       0.367210
max       0.809095
Name: trend_score, dtype: float64

In [386]:


conn = sqlite3.connect(WORK_DB)

df_latest.to_sql("video_trends", conn, if_exists="replace", index=False)

conn.close()


In [387]:
import sqlite3

conn = sqlite3.connect(WORK_DB)

tables = conn.execute(
    "SELECT name FROM sqlite_master WHERE type='table';"
).fetchall()

conn.close()

print(tables)


[('video_stats',), ('video_features',), ('video_viral_status',), ('video_trends',)]


In [388]:
conn = sqlite3.connect(WORK_DB)

df = pd.read_sql("SELECT * FROM video_features", conn)

conn.close()


In [389]:
df["collected_at"] = pd.to_datetime(df["collected_at"], utc=True)

df_latest = (
    df.sort_values("collected_at")
      .groupby("video_id")
      .tail(1)
      .reset_index(drop=True)
)


In [390]:
viral_threshold = df_latest["view_velocity"].quantile(0.90)

print("Viral threshold:", viral_threshold)


Viral threshold: 43093.38514128562


In [391]:
df_latest["is_viral"] = (df_latest["view_velocity"] >= viral_threshold).astype(int)
df_latest["viral_score"] = df_latest["view_velocity"] / viral_threshold

df_latest["is_viral"].value_counts()


is_viral
0    45
1     5
Name: count, dtype: int64

In [392]:
viral_now = df_latest[df_latest["is_viral"] == 1]

viral_now = viral_now.sort_values("view_velocity", ascending=False)

viral_now[[
    "title",
    "channel",
    "view_velocity",
    "engagement_rate",
    "age_hours",
    "viral_score"
]].head(10)


,title,channel,view_velocity,engagement_rate,age_hours,viral_score
39,Asla (Official Video) Mankirt Aulakh | Zehr Vi...,Mankirt Aulakh,148747.356931,0.016861,8.498603,3.451745
33,Bridgerton Season 4 | Part 2 Official Trailer ...,Netflix,89634.167486,0.064347,5.099715,2.079998
40,Poppy Playtime: Chapter 5 - Official Game Trailer,Mob Entertainment,58942.970327,0.076185,50.096659,1.367796
2,Enakena Yaarum Illaye Promo | LIK | Pradeep Ra...,Sony Music South,50104.227582,0.089797,8.179437,1.162690
48,Untitled John Wick Game - Announce Trailer | P...,PlayStation,46760.802056,0.077482,21.197215,1.085104


In [393]:
conn = sqlite3.connect(WORK_DB)

df_latest.to_sql("video_viral_status", conn, if_exists="replace", index=False)

conn.close()


In [394]:
###Early Viral Detection###

In [395]:
from googleapiclient.discovery import build
from datetime import datetime, timezone, timedelta
import pandas as pd

API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=API_KEY)

# def fetch_recent(region="IN"):
#     now = datetime.now(timezone.utc)
    
#     # Look back 12 hours (6 can be too narrow)
#     published_after = (now - timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
#     request = youtube.search().list(
#         part="snippet",
#         q="news",                 # Broad query to capture most videos
#         order="date",
#         publishedAfter=published_after,
#         type="video",
#         regionCode=region,
#         maxResults=100
#     )
    
#     response = request.execute()
    
#     video_ids = [
#         item["id"]["videoId"]
#         for item in response["items"]
#         if item["id"]["kind"] == "youtube#video"
#     ]
    
#     return video_ids
def fetch_recent(region="IN"):
    """Fetch videos from multiple search keywords for diversity"""
    all_video_ids = []
    
    # Define diverse search keywords to get different types of videos
    search_keywords = [
        "trending",      # Trending videos
        "viral",         # Viral content
        "music",         # Music videos
        "gaming",        # Gaming videos
        "technology",    # Tech content
        "entertainment", # Entertainment
        "news",          # News (original)
        "sports",        # Sports
        "education",     # Educational
        "lifestyle"      # Lifestyle
    ]
    
    now = datetime.now(timezone.utc)
    published_after = (now - timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    for keyword in search_keywords:
        try:
            request = youtube.search().list(
                part="snippet",
                q=keyword,
                order="date",
                publishedAfter=published_after,
                type="video",
                regionCode=region,
                maxResults=100  # ← Changed from 50 to 100
            )
            response = request.execute()
            
            video_ids = [
                item["id"]["videoId"] 
                for item in response["items"] 
                if item["id"]["kind"] == "youtube#video"
            ]
            all_video_ids.extend(video_ids)
            print(f"✓ Fetched {len(video_ids)} videos for keyword: '{keyword}'")
            
        except Exception as e:
            print(f"⚠ Error fetching '{keyword}': {str(e)}")
            continue
    
    print(f"\n✓ Total video IDs collected: {len(all_video_ids)}")
    return all_video_ids


In [396]:
# def fetch_video_stats(video_ids):
#     request = youtube.videos().list(
#         part="snippet,statistics",
#         id=",".join(video_ids)
#     )
    
#     response = request.execute()
    
#     now = datetime.now(timezone.utc)
#     data = []
    
#     for item in response["items"]:
#         stats = item["statistics"]
#         snippet = item["snippet"]
        
#         data.append({
#             "video_id": item["id"],
#             "title": snippet["title"],
#             "channel": snippet["channelTitle"],
#             "published_at": snippet["publishedAt"],
#             "views": int(stats.get("viewCount", 0)),
#             "likes": int(stats.get("likeCount", 0)),
#             "comments": int(stats.get("commentCount", 0)),
#             "collected_at": now,
#             "source": "search"
#         })
    
#     return pd.DataFrame(data)


def fetch_video_stats(video_ids):
    """Fetch video statistics in batches of 50 (API limit)"""
    
    all_data = []
    batch_size = 50
    
    # Split video_ids into batches of 50
    for i in range(0, len(video_ids), batch_size):
        batch = video_ids[i:i+batch_size]
        print(f"Fetching stats for batch {i//batch_size + 1}: {len(batch)} videos")
        
        try:
            request = youtube.videos().list(
                part="snippet,statistics",
                id=",".join(batch)  # Join max 50 IDs
            )
            response = request.execute()
            
            now = datetime.now(timezone.utc)
            
            for item in response["items"]:
                stats = item["statistics"]
                snippet = item["snippet"]
                
                all_data.append({
                    "video_id": item["id"],
                    "title": snippet["title"],
                    "channel": snippet["channelTitle"],
                    "published_at": snippet["publishedAt"],
                    "views": int(stats.get("viewCount", 0)),
                    "likes": int(stats.get("likeCount", 0)),
                    "comments": int(stats.get("commentCount", 0)),
                    "collected_at": now,
                    "source": "search"
                })
        except Exception as e:
            print(f"⚠ Error in batch {i//batch_size + 1}: {str(e)}")
            continue
    
    print(f"✓ Total video stats fetched: {len(all_data)}")
    return pd.DataFrame(all_data)



In [397]:
video_ids = fetch_recent()
print("Found videos:", len(video_ids))
print(video_ids[:5])


✓ Fetched 50 videos for keyword: 'trending'
✓ Fetched 50 videos for keyword: 'viral'
✓ Fetched 50 videos for keyword: 'music'
✓ Fetched 50 videos for keyword: 'gaming'
✓ Fetched 50 videos for keyword: 'technology'
✓ Fetched 50 videos for keyword: 'entertainment'
✓ Fetched 50 videos for keyword: 'news'
✓ Fetched 50 videos for keyword: 'sports'
✓ Fetched 50 videos for keyword: 'education'
✓ Fetched 50 videos for keyword: 'lifestyle'

✓ Total video IDs collected: 500
Found videos: 500
['drTECrsEn08', 'SlqJVm6G64E', 'M4M-jpY1RNo', 'lUwJP3JjS3Q', 'uMBxx0jDuxw']


In [398]:
# conn = sqlite3.connect("youtube.db")

# conn.execute("ALTER TABLE video_stats ADD COLUMN source TEXT")

# conn.commit()
# conn.close()

# print("Column 'source' added")


In [399]:
df_new = fetch_video_stats(video_ids)

print("Rows fetched:", len(df_new))
df_new.head()



Fetching stats for batch 1: 50 videos
Fetching stats for batch 2: 50 videos
Fetching stats for batch 3: 50 videos
Fetching stats for batch 4: 50 videos
Fetching stats for batch 5: 50 videos
Fetching stats for batch 6: 50 videos
Fetching stats for batch 7: 50 videos
Fetching stats for batch 8: 50 videos
Fetching stats for batch 9: 50 videos
Fetching stats for batch 10: 50 videos
✓ Total video stats fetched: 500
Rows fetched: 500


,video_id,title,channel,published_at,views,likes,comments,collected_at,source
0,drTECrsEn08,Saleem Sir Is The Best Physics Teacher🔥🛐 #shor...,IIT Roorkee,2026-02-16T17:25:00Z,3127,0,0,2026-02-16 17:51:55.176037+00:00,search
1,SlqJVm6G64E,Namaz Aur Shaitan😨🤣 #shorts #trending #islamic...,Adeeb Khushi 06,2026-02-16T17:16:33Z,56940,868,0,2026-02-16 17:51:55.176037+00:00,search
2,M4M-jpY1RNo,नारियल पानी वाला😱😱 #trending #food #youtubeshorts,Nariyal Ka Swad,2026-02-16T17:05:50Z,1363,55,0,2026-02-16 17:51:55.176037+00:00,search
3,lUwJP3JjS3Q,Chicken 🍗❤️ #chicken #chickencurry #desichicke...,COOKINGxSHOT,2026-02-16T16:49:08Z,1725,0,0,2026-02-16 17:51:55.176037+00:00,search
4,uMBxx0jDuxw,Dananeer’s best #trending #ytshorts #emotional...,Haleema Salman,2026-02-16T16:27:33Z,10386,341,1,2026-02-16 17:51:55.176037+00:00,search


In [400]:
import sqlite3
# Check existing video IDs
# Cell 338: Check and filter duplicates
conn = sqlite3.connect(WORK_DB)
existing_ids = set(pd.read_sql("SELECT DISTINCT video_id FROM video_stats", conn)['video_id'].tolist())
conn.close()

df_new_filtered = df_new[~df_new['video_id'].isin(existing_ids)]

# Only ONE append statement below:
if len(df_new_filtered) > 0:
    conn = sqlite3.connect(WORK_DB)
    df_new_filtered.to_sql("video_stats", conn, if_exists="append", index=False)
    conn.close()
    print(f"✓ Added {len(df_new_filtered)} new unique videos")
else:
    print("⚠ No new unique videos found!")




conn = sqlite3.connect(WORK_DB)
df_new.to_sql("video_stats", conn, if_exists="append", index=False)
conn.close()

print("New videos saved:", len(df_new))


✓ Added 141 new unique videos
New videos saved: 500


In [401]:
conn = sqlite3.connect(WORK_DB)
total_rows = pd.read_sql("SELECT COUNT(*) as count FROM video_stats", conn)['count'][0]
unique_videos = pd.read_sql("SELECT COUNT(DISTINCT video_id) as count FROM video_stats", conn)['count'][0]
conn.close()
print(f"Total rows: {total_rows}")
print(f"Unique videos: {unique_videos}")
print(f"Rows per video (average): {total_rows / unique_videos:.1f}")


Total rows: 2492
Unique videos: 749
Rows per video (average): 3.3


In [402]:
conn = sqlite3.connect(WORK_DB)
df = pd.read_sql("SELECT * FROM video_features", conn)
conn.close()

df["collected_at"] = pd.to_datetime(df["collected_at"], utc=True)

df_latest = (
    df.sort_values("collected_at")
      .groupby("video_id")
      .tail(1)
)


In [403]:
early_videos = df_latest[df_latest["age_hours"] <= 24]


In [404]:
threshold_early = early_videos["view_velocity"].quantile(0.90)

early_videos["early_viral"] = (
    early_videos["view_velocity"] >= threshold_early
).astype(int)


/tmp/ipykernel_55/1131338104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  early_videos["early_viral"] = (


In [405]:
early_candidates = early_videos[early_videos["early_viral"] == 1]

early_candidates.sort_values(
    "view_velocity", ascending=False
)[[
    "title",
    "channel",
    "view_velocity",
    "age_hours"
]].head(15)


,title,channel,view_velocity,age_hours
629,Asla (Official Video) Mankirt Aulakh | Zehr Vi...,Mankirt Aulakh,148747.356931,8.498603
653,Bridgerton Season 4 | Part 2 Official Trailer ...,Netflix,89634.167486,5.099715
570,Enakena Yaarum Illaye Promo | LIK | Pradeep Ra...,Sony Music South,50104.227582,8.179437


In [406]:
###Viral Prediction###

In [407]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(WORK_DB)
df = pd.read_sql("SELECT * FROM video_features", conn)
conn.close()

df["collected_at"] = pd.to_datetime(df["collected_at"], utc=True)
df = df.sort_values(["video_id", "collected_at"])


In [408]:
early_snap = df.groupby("video_id").head(1)
late_snap  = df.groupby("video_id").tail(1)

print("Early:", len(early_snap))
print("Late:", len(late_snap))



Early: 50
Late: 50


In [409]:
valid_videos = late_snap[late_snap["age_hours"] >= 24]["video_id"]

early_snap = early_snap[early_snap["video_id"].isin(valid_videos)]
late_snap  = late_snap[late_snap["video_id"].isin(valid_videos)]

print("Videos with future data:", len(valid_videos))



Videos with future data: 26


In [410]:
late_threshold = late_snap["view_velocity"].quantile(0.90)
print("Future viral threshold:", late_threshold)

late_snap["future_viral"] = (
    late_snap["view_velocity"] >= late_threshold
).astype(int)


Future viral threshold: 13952.217078141675


In [411]:
train_df = early_snap.merge(
    late_snap[["video_id", "future_viral"]],
    on="video_id",
    how="inner"
)

print("Training samples:", len(train_df))
train_df["future_viral"].value_counts()


Training samples: 26


future_viral
0    23
1     3
Name: count, dtype: int64

In [412]:
features = [
    "view_velocity",
    "acceleration",
    "engagement_rate",
    "age_hours",
    "delta_views"
]

X = train_df[features]
y = train_df["future_viral"]


In [413]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",
    random_state=42
)
model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [414]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.00      0.00      0.00         1

    accuracy                           0.83         6
   macro avg       0.42      0.50      0.45         6
weighted avg       0.69      0.83      0.76         6



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [415]:
print("Training samples:", len(train_df))
print(train_df["future_viral"].value_counts())


Training samples: 26
future_viral
0    23
1     3
Name: count, dtype: int64


In [416]:
print("Predictions:", y_pred)
print("Actual:", y_test.values)


Predictions: [0 0 0 0 0 0]
Actual: [0 0 0 0 0 1]


In [417]:
conn = sqlite3.connect(WORK_DB)
df = pd.read_sql("SELECT collected_at FROM video_features", conn)
conn.close()

df['collected_at'] = pd.to_datetime(df['collected_at'], utc=True)

print("Latest timestamp:", df['collected_at'].max())


Latest timestamp: 2026-02-13 20:05:58.972205+00:00
